# Anime Generation
## DCGAN 

> Data pre-processing [here](https://www.tensorflow.org/tutorials/load_data/images)

In [ ]:
from IPython.display import Audio
from IPython import display
the_voice = './sounds/chose_a_voice.wav'
the_creation = './sounds/You_created_me.wav'

import tensorflow as tf
# Error: Node: 'sequential_3/dropout_4/dropout/random_uniform/RandomUniform'
# OOM when allocating tensor with shape[256,180,120,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
# 	 [[{{node sequential_3/dropout_4/dropout/random_uniform/RandomUniform}}]]
# the solution to thi is report_tensor_allocations_upon_oom = True
run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom=True)
runmeta = tf.compat.v1.RunMetadata()
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.distribute.MultiWorkerMirroredStrategy(
    cluster_resolver=None, communication_options=None
)

import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['WANDB_NOTEBOOK_NAME'] = 'Anime Faces and Names'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import time
from tensorflow import keras
from tensorflow.keras import layers
import glob
import matplotlib.pyplot as plt
import numpy as np
import imageio
import tensorflow_docs
import PIL
import wandb
# from train.dcgan import DCGAN
# pip install wandb
# wandb login
# 65bacd21c40b0085e299e05ea94b552d5119b7bc

# wandb.init(project="Anime Creation", entity="skycladai")

print("Note! You will need tensorflow-gpu version 2.8.0 or higher. Your version is", tf.__version__)

Audio(url=the_voice, autoplay=True, rate=48000)

> Configuration

In [ ]:
# BATCH_SIZE = 254 # or 64 this cause memory allocation failure
BATCH_SIZE = 8
IMAGE_WIDTH = 240
IMAGE_LENGTH = 360
ORIGINAL_IMAGE_SIZE = (360, 240)
RGB = 3
BUFFER_SIZE = 58085
EPOCHS = 80
noise_dim = 100
num_examples_to_generate = 9
wandb.config = {
  "learning_rate": 0.001,
  "epochs": EPOCHS,
  "batch_size": BATCH_SIZE
}
# dcgan = DCGAN(batch_size= BATCH_SIZE, n_noise= noise_dim, num_examples_to_generate=num_examples_to_generate)


# Load Dataset

In [ ]:
%%time

x_train = tf.keras.preprocessing.image_dataset_from_directory( directory="./dataset/", validation_split=0.3, subset="training" ,label_mode=None, batch_size=BATCH_SIZE, image_size=ORIGINAL_IMAGE_SIZE, seed=123 , shuffle=BUFFER_SIZE)
x_test = tf.keras.preprocessing.image_dataset_from_directory( directory="./dataset/", validation_split=0.3, subset="validation" ,label_mode=None, batch_size=BATCH_SIZE, image_size=ORIGINAL_IMAGE_SIZE, seed=123, shuffle=BUFFER_SIZE)
    
plt.figure(figsize=(10, 10))
plt.title("Training Images")
for images in x_train.take(1):
  print(images.shape)
  for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.axis("off")


> Cashe Dataset

In [ ]:
%%time
print("Wall time: 5min 22s")
# Cache keeps the images in memory after they're loaded off disk during the first epoch. 
# This will ensure the dataset does not become a bottleneck while training your model. 
# Prefetch overlaps data preprocessing and model execution while training.
# This allows later elements to be prepared while the current element is being processed. 
# This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
def configure_for_performance(ds, name):
    ds = ds.cache(name)
    ds = ds.shuffle(buffer_size=2000)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

normalized_ds = x_train.map(tf.keras.layers.Rescaling(1./255))
# print(normalized_ds)
# Cashe, shuffle and prefetch the data
train_ds = configure_for_performance(normalized_ds, "./cache/training_cashe")
val_ds = configure_for_performance(x_test, "./cache/testing_cashe")
for image_batch in train_ds:
    print(image_batch.shape)
    print("In range between: ", np.min(image_batch[0])," and: ", np.max(image_batch[0]))
    break


## DCGAN 
> Generator

In [ ]:
def generator_model():
    print("Generating DCGAN model.")
    model = tf.keras.Sequential()
    model.add(layers.Dense(45*30*256, use_bias=False, input_shape=(100,)))
    #     model.add(layers.BatchNormalization())
    #     model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Reshape((45, 30, 256)))
    assert model.output_shape == (None, 45, 30, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 45, 30, 128)

    #     model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 90, 60, 64)

    #     model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 180, 120, 32)

    #     model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 360, 240, 3)

    print("DCGAN model completed")
    return model

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
%%time

noise = tf.random.normal([1, 100])
generator = generator_model()
generated_image = generator(noise)
print(generated_image.shape)
plt.imshow(generated_image[0, :, :, 0])
generator.summary()

> Discriminator 

In [ ]:
def discriminator_model():
    print("DCGAN descriminator model")
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), padding='same',
                                    input_shape=[360, 240, 3]))
    model.add(layers.LeakyReLU())
    #     model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    #     model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    #     model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    #     model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1))
    print(model.output_shape)
    print("DCGAN descriminator Completed!!")
    return model

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
discriminator = discriminator_model()
decision = discriminator(generated_image)
print(decision)
discriminator.summary()

> Save Checkpoints

## Training DCGAN

### Training definitions 

In [ ]:
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

@tf.function
def train_step(images):
    print("Training step")

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        print("Generator for noise")

        generated_images = generator(noise)
        print("Discriminator for images")
        real_out = discriminator(images)
        print("Discriminator for noise")
        fake_out = discriminator(generated_images)

        gen_loss = generator_loss(fake_out)
        print("gen_loss :",gen_loss)
        disc_loss = discriminator_loss(real_output=real_out, fake_output=fake_out)
        print("disc_loss :",disc_loss)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [ ]:
def train(dataset, epochs):
    print("Training Start...")
    for epoch in range(epochs):
        print("Epoch :", epoch)
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        # Produce images for the GIF as you go
        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch + 1, seed)

        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    with tf.compat.v1.Session() as sess:
        wandb.tensorflow.log(tf.summary.merge_all())

        # Generate after the final epoch
        display.clear_output(wait=True)
        generate_and_save_images(generator, epochs, seed)

### Save and plot images

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input)

    fig = plt.figure(figsize=(10, 10))
    print(predictions)
    print(predictions.shape)
    for i in range(predictions.shape[0]):
        plt.subplot(3, 3, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5 )
        plt.axis('off')

    plt.savefig('./generated_images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

### Optimizer and Checkpoint 

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(
    generator_optimizer=generator_optimizer,
    discriminator_optimizer=discriminator_optimizer,
    generator=generator,
    discriminator=discriminator)

> Actual Train

In [ ]:
%%time

train(dataset=train_ds, epochs=EPOCHS)

Audio(url=the_creation, autoplay=True, rate=48000)

In [ ]:
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

 ## Create a GIF

In [ ]:
%%time
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('./generated_images/image_at_epoch_{:04d}.png'.format(epoch_no))
display_image(6)


In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)